# Financial Markets Expert Chatbot

# Objectives

###### Build a Stock Prediction Chatbot:
Create an AI-powered chatbot that can provide users with predictions of stock closing prices based on historical data and market trends.

###### Integrate a Time Series Predictive Model:
Integrate a Time Series predictive model to generate accurate stock price predictions.

###### User Interaction:
Enable users to interact with the chatbot through a user-friendly interface.

###### Intent Matching and Entity Extraction:
Develop intent matching to identify user queries and extract relevant entities (stock symbols and dates) from the queries.

###### Generate Responses:
Generate informative responses that provide predicted stock closing prices based on user queries.

# Fucntionalities

#### Intent Matching and Response Generation:
The chatbot matches user input with predefined intent patterns and generates appropriate responses based on the matched intent. This is handled by the get_intent and get_response functions, along with the intent data stored in the json_data dictionary.

#### Entity Extraction:
The chatbot extracts entities (stock symbols and dates) from user input using spaCy. This is achieved through the extract_entities function.

#### Stock Prediction Integration:
The chatbot integrates with a stock prediction API to fetch predicted close prices for a given stock symbol and date. This functionality is implemented in the get_stock_prediction function.

#### Streamlit Web Application:
The chatbot's interaction with users is facilitated through a Streamlit web application. Users input their queries, and the chatbot responds with relevant information. This is accomplished in the main function and the Streamlit components.

#### Predicted Close Price Logic:
The chatbot calculates the predicted close price based on the extracted entities (stock symbol and date). This logic is encapsulated within the get_predicted_close_price and fetch_predicted_close_price functions.

# Project Setup and Libraries Import



In this section, I imported the necessary libraries and modules required for building the stock prediction chatbot.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install streamlit

In [3]:
!pip install dateparser

In [4]:
import os
import pandas as pd
import json
import numpy as np

import dateparser
import re
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

import streamlit as st
import spacy
import random

import requests
import datetime

import streamlit as st


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Predefined Intents and Data Preparation
In this section, I defined the intents for the chatbot and prepared the data for intent matching and response generation.


##### 1. Intent Matching and Response Generation:
- Functionality:
The chatbot matches user input with predefined intent patterns and generates appropriate responses based on the matched intent.

- Code Snippets:
get_intent: Uses spaCy to match user input with predefined intent patterns.
get_response: Generates chatbot responses based on matched intents and extracted entities.

In [5]:
# The json data of intents

json_data = {
  "intents": [
    {
      "tag": "greeting",
      "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Hay"],
      "responses": ["Hello, how can I help you today?", "Hi, how can I help you today?", "Hi there, how can I help you today?"]
    },
    {
      "tag": "goodbye",
      "patterns": ["Bye", "See you later", "Goodbye"],
      "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
    },
    {
      "tag": "thanks",
      "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help"],
      "responses": ["Happy to help!", "Any time!", "My pleasure", "You're most welcome!"]
    },
    {
      "tag": "about",
      "patterns": ["Who are you?", "What are you?", "Who you are?"],
      "responses": ["I'm Sammy, your close price bot pedictor, how can I help you today?", "I'm Sammy, an Artificial Intelligent bot that is capable of predicting stock close, how can I help you today?"]
    },
    {
      "tag": "name",
      "patterns": ["what is your name", "what should I call you", "whats your name?"],
      "responses": ["You can call me Sammy, how can I help you today?", "I'm Sammy!, how can I help you today", "Just call me as Sammy, how can I help you today"]
    },
    {
      "tag": "help",
      "patterns": [
        "Could you help me?",
        "give me a hand please",
        "Can you help?",
        "What can you do for me?",
        "I need a support",
        "I need a help",
        "support me please"
      ],
      "responses": [
        "Yes Sure, How can I help you?"
      ]
    },
    {
      "tag": "The closing price",
      "patterns": [
        "What is the predicted close price for {stock_symbol} on {date}?",
        "What is the forecasted close price for {stock_symbol} on {date}?",
        "What is the projected closing price of {stock_symbol} on {date}?",
        "What is the expected closing price of {stock_symbol} on {date}?",
        "What is the forecast for the closing price of {stock_symbol} on {date}?",
        "What is the predicted value of {stock_symbol} at the close of trading on {date}?",
        "What is the estimated closing price of {stock_symbol} on {date}?",
        "What is the most likely closing price of {stock_symbol} on {date}?",
        "What is the consensus closing price of {stock_symbol} on {date}?",
        "What is the best guess for the closing price of {stock_symbol} on {date}?",
        "What is the likely closing price of {stock_symbol} on {date}?",
        "What is the projected value of {stock_symbol} at the end of the trading day on {date}?",
        "What is the consensus estimate for the closing price of {stock_symbol} on {date}?",
        "What is the market's expectation for the closing price of {stock_symbol} on {date}?",
        "What is the most likely scenario for the closing price of {stock_symbol} on {date}?",
        "What is the best guess for the closing price of {stock_symbol} on {date}, given the current market conditions?",
        "What is the most likely price that {stock_symbol} will close at on {date}?",
        "What is the price that {stock_symbol} is most likely to close at on {date}, based on historical data and current market conditions?",
        "What is the price that {stock_symbol} is most likely to close at on {date}, according to my financial advisor?",
        "What is the closing price forecast for {stock_symbol} on {date}?",
        "What is the closing price outlook for {stock_symbol} on {date}?",
        "What is the closing price prediction for {stock_symbol} on {date}?",
        "What is the closing price projection for {stock_symbol} on {date}?",
        "What is the closing price estimate for {stock_symbol} on {date}?",
        "What is the closing price expectation for {stock_symbol} on {date}?",
        "What is the closing price guess for {stock_symbol} on {date}?",
        "What is the closing price anticipation for {stock_symbol} on {date}?",
        "What is the closing price speculation for {stock_symbol} on {date}?",
        "Predicted close price for {stock_symbol} on {date}?",
        "Tell me the predicted close price of {stock_symbol} on {date}",
        "Predicted close price of {stock_symbol} on {date}",
        "Forecasted close price for {stock_symbol} on {date}",
        "Projected closing price of {stock_symbol} on {date}",
        "Expected closing price of {stock_symbol} on {date}",
        "Forecast for the closing price of {stock_symbol} on {date}",
        "Predicted value of {stock_symbol} at the close of trading on {date}",
        "Estimated closing price of {stock_symbol} on {date}",
        "Most likely closing price of {stock_symbol} on {date}",
        "Consensus closing price of {stock_symbol} on {date}",
        "Best guess for the closing price of {stock_symbol} on {date}",
        "Likely closing price of {stock_symbol} on {date}",
        "Projected value of {stock_symbol} at the end of the trading day on {date}",
        "Consensus estimate for the closing price of {stock_symbol} on {date}",
        "Market's expectation for the closing price of {stock_symbol} on {date}",
        "Most likely scenario for the closing price of {stock_symbol} on {date}",
        "Best guess for the closing price of {stock_symbol} on {date}, given the current market conditions",
        "Most likely price that {stock_symbol} will close at on {date}",
        "Price that {stock_symbol} is most likely to close at on {date}, based on historical data and current market conditions",
        "Price that {stock_symbol} is most likely to close at on {date}, according to my financial advisor",
        "Closing price forecast for {stock_symbol} on {date}",
        "Closing price outlook for {stock_symbol} on {date}",
        "Closing price prediction for {stock_symbol} on {date}",
        "Closing price projection for {stock_symbol} on {date}",
        "Closing price estimate for {stock_symbol} on {date}",
        "Closing price expectation for {stock_symbol} on {date}",
        "Closing price guess for {stock_symbol} on {date}",
        "Closing price anticipation for {stock_symbol} on {date}",
        "Closing price speculation for {stock_symbol} on {date}"
      ],
      "responses": [
        "The closing price prediction for {stock_symbol} on {date} is {predicted_close}. Please note that this prediction is based on historical data and market trends and may not be entirely accurate. Stock prices can be volatile and subject to various factors, so it's essential to consider this information as a reference and consult with a financial advisor for informed investment decisions."
      ]
    }
  ]
}
# Construct the full directory path
relative_directory = "/content/drive/MyDrive/ColabNotebooks/stock_price_pred_chatbot-main/stock_price_pred_chatbot-main"
output_directory = os.path.expanduser(os.path.join("~", relative_directory))

# Create the full file path
file_path = os.path.join(output_directory, "/content/drive/MyDrive/ColabNotebooks/stock_price_pred_chatbot-main/stock_price_pred_chatbot-main/intents_data.json")

# Write the JSON data to the specified file path
with open(file_path, "w") as json_file:
    json.dump(json_data, json_file, indent=4)

print(f"JSON file saved to: {file_path}")


JSON file saved to: /content/drive/MyDrive/ColabNotebooks/stock_price_pred_chatbot-main/stock_price_pred_chatbot-main/intents_data.json


In [6]:
csv_data = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/stock_price_pred_chatbot-main/stock_price_pred_chatbot-main/stocks  predicted closing prices Datasets/preprocessed_close_price_predictions.csv')

# Intent Matching with spaCy
This section involves using spaCy to perform intent matching based on user input and predefined patterns.

In [7]:
import spacy

# Load spaCy's English language model
nlp = spacy.load("en_core_web_sm")

def get_intent(user_input, intents):
    doc = nlp(user_input)
    matched_intent = None
    for intent in intents:
        for pattern in intent['patterns']:
            pattern_doc = nlp(pattern)
            if pattern_doc.similarity(doc) > 0.7:  # Similarity threshold can be adjusted as required
                matched_intent = intent
                break
        if matched_intent:
            break
    return matched_intent

# Testing
intents = json_data['intents']  # Having loaded the intents from JSON

user_input = input("User: ")
matched_intent = get_intent(user_input, intents)

if matched_intent:
    print("Matched intent:", matched_intent['tag'])
else:
    print("No matching intent found.")


User: Hi
Matched intent: greeting


# Entity Extraction with spaCy
Here, I used spaCy to extract entities (stock symbol and date) from user input.



##### 2. Entity Extraction:
- Functionality: The chatbot extracts entities (stock symbols and dates) from user input using spaCy.

- Code Snippet:
extract_entities: Utilizes spaCy to extract entities (stock symbols and dates) from user input.

In [8]:
def extract_entities(user_input):
    doc = nlp(user_input)
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))
    return entities

# Testing
user_input = "What is the predicted close price for AAPL on 2023-08-16?"
entities = extract_entities(user_input)
for text, label in entities:
    print(f"Entity: {text}, Label: {label}")


Entity: 2023-08-16, Label: DATE


# Generating Chatbot Responses
This section focuses on generating chatbot responses based on the matched intent and extracted entities.

In [9]:
# Load spaCy's English language model
nlp = spacy.load("en_core_web_sm")

# Load intents data from the JSON file
with open('/content/drive/MyDrive/ColabNotebooks/stock_price_pred_chatbot-main/stock_price_pred_chatbot-main/intents_data.json', 'r') as f:
    intents = json.load(f)

def extract_entities(user_input):
    doc = nlp(user_input)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

def get_response(user_input, entities):
    user_input_lower = user_input.lower()
    response = "I'm sorry, I don't understand. Can you please rephrase or provide more details?"

    for intent in intents:
        for pattern in intent['patterns']:
            if pattern.lower() in user_input_lower:
                response = random.choice(intent['responses'])
                if '{stock_symbol}' in response and '{date}' in response and '{predicted_close}' in response:
                    for entity_text, entity_label in entities:
                        if entity_label == 'STOCK_SYMBOL':
                            response = response.replace('{stock_symbol}', entity_text)
                        elif entity_label == 'DATE':
                            response = response.replace('{date}', entity_text)

                    # Get the predicted close price based on stock_symbol and date
                    predicted_close = get_predicted_close_price(entities)  # For logic
                    response = response.replace('{predicted_close}', str(predicted_close))
                break

    return response


# Model CSV Integration with Streamlit
In this section, I integrated the dataset from the generated output of the model to fetch stock predictions based on user input as well as handles the logic for fetching the predicted close price using the extracted entities.


- Functionality: The chatbot integrates with the Predictive Model to fetch predicted close prices for a given stock symbol and date. It the calculate the predicted close price based on the extracted entities (stock symbol and date).
- Code Snippet: Implements the integration of the Model and logic to fetch predicted close prices from the prediction source (Model CSV)for a desired stock symbol and date.
- Streamlit Web Application:
    Functionality: The chatbot's interaction with users is facilitated through a Streamlit web application. Users input their queries, and the chatbot responds with relevant information.
    Code Snippet: main: Creates a Streamlit web application for user interaction and displays the chatbot's responses.

In [10]:
def get_predicted_close_price(csv_data, stock_symbol, date):
    try:
        data_row = csv_data[(csv_data['stock_symbol'] == stock_symbol) & (csv_data['date'] == date)]
        if not data_row.empty:
            predicted_close = data_row.iloc[0]['predicted_close']
            return predicted_close
        else:
            return "No data available for the specified stock symbol and date."
    except Exception as e:
        return "An error occurred while retrieving data: " + str(e)

# Running the Streamlit Application
Lastly, this section initiates the execution of the Streamlit application.

In [11]:
import streamlit as st

# Your modified functions and data loading here...

st.title("Stock Price Prediction Chatbot")

user_input = st.text_input("You: ")
if st.button("Submit"):
    matched_intent = get_intent(user_input, intents)
    if matched_intent:
        entities = extract_entities(user_input)
        response = get_response(user_input, entities)
    else:
        response = "No matching intent found."
    st.write("Bot:", response)

2023-08-18 07:29:43.057 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py [ARGUMENTS]


In [ ]:
!streamlit run /content/drive/MyDrive/ColabNotebooks/stock_price_pred_chatbot-main/stock_price_pred_chatbot-main/stock_price_predictor.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.245.59.60:8501



# Summary
The project involves the creation of a stock prediction chatbot named "Sammy." Leveraging a blend of natural language processing, machine learning, and web development tools, the chatbot serves as a user-friendly platform for obtaining predicted stock closing prices.

The chatbot's core functionalities include intent matching, where user queries are matched with predefined patterns, and entity extraction, which identifies relevant information such as stock symbols and dates. By integrating with a stock prediction API, the chatbot fetches and provides accurate predicted close prices based on user input.

A Streamlit web application acts as the interface, enabling users to input queries and receive informative responses. The application seamlessly interacts with the chatbot, showcasing its ability to understand user intent, extract data, and deliver contextually relevant predictions.

The project demonstrates the successful integration of various technologies, such as spaCy for natural language processing, TensorFlow for machine learning, scikit-learn for data preprocessing, and Streamlit for web deployment. By combining these elements, the chatbot offers a practical solution for users seeking stock market insights and predictions.